# Assignment 2 (Hopfield Network)

## 1) Storing the weights

In [22]:
import numpy as np
import pandas as pd

class HopfieldNetwork:
    """
    Generic network, initiate with array of patterns
    number_of_weights = (n*(n-1))/2
    """
    def __init__(self, patterns):
        self.patterns = patterns
        self.weights = self.evaluate_weights()
        self.stable_patterns = self.evaluate_stable_patterns()
 
    def initiate_weights(self):
        n = len(self.patterns[0])
        weights = {}
        for index in range(n):
            first_index = index + 1 # to make the 0 become 1 and so on
            second_index = first_index + 1
            while second_index <= n:
                weight_id = "{},{}".format(first_index, second_index)
                weights[weight_id] = "yoo"
                second_index = second_index + 1
        return weights

    def calculate_weight_from_patterns(self,unit_1_index, unit_2_index):
        multiplicants = [pattern[unit_1_index] * pattern[unit_2_index] for pattern in patterns]
        return np.array(multiplicants).sum()

    def evaluate_weights(self):
        weights = self.initiate_weights()
        for weight_key in weights.keys():
            unit_1_index = int(weight_key.split(",")[0]) - 1
            unit_2_index = int(weight_key.split(",")[1]) - 1
            calculated_weight = self.calculate_weight_from_patterns(unit_1_index, unit_2_index)
            weights[weight_key] = calculated_weight
        return weights

    def state_update(self, state_index, pattern):
        current_state_index = str(state_index+1)
        relevant_weights = {k: v for k, v in self.weights.items() if current_state_index in k.split(",")}
        multiplications = []
        for relevant_weight_key, relevant_weight_value in relevant_weights.items():
            other_state = [idx for idx in relevant_weight_key.split(",") if idx != current_state_index][0]
            other_state_index = int(other_state) - 1
            multiplications.append(pattern[other_state_index] * relevant_weight_value)
        return np.sign(np.array(multiplications).sum())
    
    def evaulate_pattern_stability(self, pattern):
        """
        Returns True if every state is stable for this pattern
        False otherwise
        """
        matches = []
        print("====================")
        print("Input Pattern: {}".format(pattern))
        evaluated_pattern = []
        for state_index, state_value in enumerate(pattern):
            updated_state = self.state_update(state_index, pattern)
            evaluated_pattern.append(updated_state)
            matches.append(updated_state == state_value)
        print("Evaluated Pattern: {}".format(evaluated_pattern))
        matched = evaluated_pattern == pattern
        print("stable: {}".format(matched))
        return matched

    def evaluate_stable_patterns(self):
        stable_patterns = []
        for pattern in self.patterns:
            if self.evaulate_pattern_stability(pattern):
                stable_patterns.append(pattern)
        return stable_patterns
                

pattern_1 = [1, -1, 1, -1]
pattern_2 = [-1, 1, 1, 1]
pattern_3 = [-1, -1, -1, 1]

patterns = [pattern_1, pattern_2, pattern_3]
hopfield_network = HopfieldNetwork(patterns)
hopfield_network.weights


Input Pattern: [1, -1, 1, -1]
Evaluated Pattern: [1, -1, 1, -1]
stable: True
Input Pattern: [-1, 1, 1, 1]
Evaluated Pattern: [-1, 1, -1, 1]
stable: False
Input Pattern: [-1, -1, -1, 1]
Evaluated Pattern: [-1, 1, -1, 1]
stable: False


{'1,2': -1, '1,3': 1, '1,4': -3, '2,3': 1, '2,4': 1, '3,4': -1}

### Summary

Only the pattern [1, -1, 1, -1] **(pattern A)** is stable. Tested this by 
1. Updating the state for each unit using the given function
2. Tested if the "pattern state" matches the "evaluated state"
3. If 2 is True for all states, then a pattern is stable

### 2) What does the network converge to in states A/B/C

- **Pattern A** Converges to itself
- **Pattern B** Converges to [-1, 1, -1, 1]
- **Pattern C** Converges to [-1, 1, -1, 1]